1) Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)
Логин тестового ящика: study.ai_172@mail.ru
Пароль тестового ящика: NextPassword172???

In [124]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import selenium.common.exceptions as s_exceptions
from selenium.webdriver.common.keys import Keys


In [125]:
chrome_options = Options()
chrome_options.add_argument("--window-size=900,1000")

driver = webdriver.Chrome(executable_path="./chromedriver.exe", options=chrome_options)
driver.get('https://mail.ru/')
driver.implicitly_wait(10)

elem = driver.find_element_by_class_name("email-input")
elem.send_keys('study.ai_172@mail.ru')

elem = driver.find_element_by_class_name("button")
elem.click()

elem = driver.find_element_by_class_name("password-input")
elem.send_keys('NextPassword172???')

elem = driver.find_element_by_class_name("second-button")
elem.click()


In [126]:
# сбор ссылок на письма
links_set=set()
last_item = None
while True:
	link = driver.find_elements_by_class_name('llc')
	new_links = [i.get_attribute('href').split('?')[0] for i in link]
	actions = ActionChains(driver)
	driver.execute_script("arguments[0].scrollIntoView();", link[-1])
	if last_item == link[-1]:
		break
	links_set.update(new_links)
	last_item = link[-1]
	time.sleep(1)

In [127]:
# сбор данных из писем
links_list = list(links_set)
messages = []

for i in links_list:

	info = {}
	driver.get(i)

	info['link'] = i
	info['sender'] = driver.find_element_by_class_name("letter-contact").text #от кого
	info['senders_email'] = driver.find_element_by_class_name("letter-contact").get_attribute('title') #от кого_email
	info['date'] = driver.find_element_by_class_name("letter__date").text	#дата отправки
	info['subject'] = driver.find_element_by_class_name("thread__subject").text #тема письма
	info['content'] = driver.find_element_by_class_name("letter-body").text #текст письма полный

	messages.append(info)


In [132]:
df = pd.DataFrame(messages)
df.head(10)

,sender,senders_email,date,subject,content,link
0,Пульс,noreply@pulse.mail.ru,"30 августа, 20:17",Топ новостей за 24 часа для вас,Новости за 24 часа\n Привет! Это ваша...,https://e.mail.ru/inbox/0:16303366600292109557:0/
1,WMJ,digest@email.wmj.ru,"28 июля, 12:26",Как коронавирус влияет на слух,Привет! Самые актуальные новости и любопытные ...,https://e.mail.ru/inbox/0:16274572001024318996:0/
2,Mail.ru,security@id.mail.ru,"15 июля, 19:40",Изменён пароль в аккаунте,Изменён пароль в аккаунте\n study.ai_1...,https://e.mail.ru/inbox/0:16263600431724643125:0/
3,Instagram,no-reply@mail.instagram.com,"14 июля, 9:12","study.ai_172, посмотрите новости TONI GARRN и ...","Подпишитесь на обновления TONI GARRN, E! New...",https://e.mail.ru/inbox/0:16262359501538944543:0/
4,Пульс,noreply@pulse.mail.ru,"4 июля, 20:16",Топ новостей за 24 часа для вас,Новости за 24 часа\n Привет! Это ваша...,https://e.mail.ru/inbox/0:16254117830033865031:0/
5,Webdesigner News,no-reply@webdesignernews.com,"30 июля, 22:34","✏ Developer Ecosystem State, How To Be An Illu...",Master The Basics of Web Design With This On...,https://e.mail.ru/inbox/0:16276664761743324844:0/
6,Пульс,noreply@pulse.mail.ru,"27 июля, 20:16",Топ новостей за 24 часа для вас,Новости за 24 часа\n Привет! Это ваша...,https://e.mail.ru/inbox/0:16273989911710944086:0/
7,WMJ,digest@email.wmj.ru,"12 июля, 11:47",Умер участник шоу «Голос. Дети»,Привет! Самые актуальные новости и любопытные ...,https://e.mail.ru/inbox/0:16260724701189065882:0/
8,Пульс,noreply@pulse.mail.ru,"26 июля, 20:17",Топ новостей за 24 часа для вас,Новости за 24 часа\n Привет! Это ваша...,https://e.mail.ru/inbox/0:16273126660164666416:0/
9,GeekBrains,noreply@e.geekbrains.ru,"26 апреля, 17:43",Удача: 4 курса в подарок + скидки на более 50 ...,Привет!\n Впереди майские праздники — с...,https://e.mail.ru/inbox/0:16194409941621379768:0/


In [134]:

from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
letters = client.letters
db = letters.mailru_messages

for i in range(0, len(df)):
	# db.insert_one(data.loc[i].to_dict())
	db.update_one({'link': df.loc[i].to_dict()['link']}, {
		'$set': df.loc[i].to_dict()}, upsert=True)